In [1]:

import subprocess
import os

!source /etc/network_turbo

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
        
import os
# We need Hugging Face の 镜像网站！
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"


设置成功
注意：仅限于学术用途，不承诺稳定性保证


In [2]:
from datasets import load_dataset
import time
start_time = time.time()
train_dataset = load_dataset("json", data_files="data/translation2019zh_train.json")
end_time = time.time()
print(f"加载train集耗时: {end_time - start_time:.2f} 秒")
valid_dataset = load_dataset("json", data_files="data/translation2019zh_valid.json")



加载train集耗时: 1.01 秒


In [3]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['english', 'chinese'],
        num_rows: 5161434
    })
})

In [4]:
split_datasets = train_dataset["train"].train_test_split(test_size=0.1, seed=42)
split_datasets["train"][1]

{'english': 'My fervor is the decoration in the decadent native place blurred flowers, the surface magnificently with making noise, has hidden too many despairs.',
 'chinese': '我的激情是装饰的花卉，颓废的本土模糊了表面的华丽与喧嚣，隐藏了太多的绝望。'}

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline



model_name = "FINGU-AI/QWEN2.5-7B-Bnk-5e"
translator = pipeline("translation", model=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

max_length = 1024
def preprocess_function(examples):
    inputs = [ex for ex in examples["english"]]
    targets = [ex for ex in examples["chinese"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

import evaluate

metric = evaluate.load("sacrebleu")